Install gwnet package:

In [ ]:
!pip install --upgrade git+git://github.com/Sunnesoft/g2net-challenge.git

from gwnet import CQTProcessor, TfDevice

Mount Google Drive folder with processed by whiten, bandpass and notch filters dataset:

In [ ]:
import os
from google.colab import drive

BASE_PATH = '/content/'
DRIVE_PATH = os.path.join(BASE_PATH, 'drive')
drive.mount(DRIVE_PATH)

Set up the configuration and extract the filtered dataset from Google Drive to $BASE_PATH folder:

In [ ]:
IMAGE_SIZE = (512, 512)
BATCH_SIZE = 1024
SAMPLE_RATE = 2048
FREQ_RANGE = (50, 250, 1)
Q_RANGE = (4, 64)
TIME_RANGE = (0, 2.0, 1e-2)
MISMATCH = 0.05
INPUT_PATH = os.path.join(BASE_PATH, 'filtered/train/')
OUTPUT_PATH = os.path.join(BASE_PATH, 'cqt/train/')
ZIP_GD_IN_FILE = os.path.join(DRIVE_PATH, 'MyDrive/g2net/filtered_train.zip')
OUTPUT_ZIP_FILE = os.path.join(BASE_PATH, 'cqt_train.zip')
ZIP_GD_OUT_FILE = os.path.join(DRIVE_PATH, 'MyDrive/g2net/cqt_train.zip')

!mkdir -p $INPUT_PATH
!mkdir -p $OUTPUT_PATH
!unzip -qq $ZIP_GD_IN_FILE -d '/'

Process the filtered dataset to Constant Q-Transform spectrograms with 3-channels, where each channel corresponds to one of the LIGO-Virgo observatories:

In [ ]:
proc = CQTProcessor(mode=TfDevice.CPU)
print(proc.scan_directory(INPUT_PATH, OUTPUT_PATH, reject_if_exists=True))
proc.run(
    batch_size=BATCH_SIZE, out_path=OUTPUT_PATH, sample_rate=SAMPLE_RATE,
    qrange=Q_RANGE, mismatch=MISMATCH, time_range=TIME_RANGE,
    freq_range=FREQ_RANGE, img_size=IMAGE_SIZE, shuffle_tasks=True, verbose=True
)

Archive results and upload them to Google Drive:

In [ ]:
!zip -rq $OUTPUT_ZIP_FILE $OUTPUT_PATH
!cp $OUTPUT_ZIP_FILE $ZIP_GD_OUT_FILE
